In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

import jax
import jax.numpy as jnp
import ilqrx.integrators as integrators
jax.config.update("jax_enable_x64", True)
print(jax.devices())

[CudaDevice(id=0)]


In [2]:
from modeling.dynamics.quadrotor import Quadrotor
from modeling.costs.lqr_cost import LQRCostWithQuaternion

dt = 0.01

quadrotor_model = Quadrotor(dt=dt, integrator=integrators.rk4)
dynamics = quadrotor_model.dynamics_dt

nx = quadrotor_model.state_dim
nu = quadrotor_model.control_dim
ndx = quadrotor_model.tangent_space_dim

T = 200

print(f"State dim: {nx}, Control dim: {nu}, Tangent space dim: {ndx}")

x_target = jnp.array([
    2., 3., 4., 
    1., 0., 0., 0.,
    0., 0., 0.,
    0., 0., 0.
])
u_target = jnp.array([9.81, 0., 0., 0.])

Q = jnp.diag(jnp.array([
    10., 10., 10.,
    100., 100., 100., 100.,
    1., 1., 1.,
    1., 1., 1.
]))
Qf = 50 * Q
R = jnp.eye(nu)

cost_model = LQRCostWithQuaternion(
    Q = Q,
    R = R,
    Qf = Qf,
    x_ref = x_target,
    u_ref = u_target,
    T = T,
    quat_start_index=3)

State dim: 13, Control dim: 4, Tangent space dim: 12


In [3]:
import ilqrx.quat_utils as quat_utils
from ilqrx.abstract_constraint import AbstractConstraint

x0 = jnp.array([
    0., 0., 0.,
    1., 0., 0., 0.,
    0., 0., 0.,
    0., 0., 0.
])
U = jnp.zeros((T, nu))
U = U.at[:].set(u_target)

# linear interpolation in position, slerp in quaternion
X = jnp.zeros((T+1, nx))
pos_start = x0[0:3]
pos_end = x_target[0:3]
quat_start = x0[3:7]
quat_end = x_target[3:7]
for k in range(T+1):
    alpha = k / T
    pos_k = (1 - alpha) * pos_start + alpha * pos_end
    quat_k = quat_utils.quat_slerp(quat_start, quat_end, alpha)
    vel_k = jnp.zeros(3)
    omega_k = jnp.zeros(3)
    x_k = jnp.concatenate([pos_k, quat_k, vel_k, omega_k])
    X = X.at[k].set(x_k)

obstacles = [
    {'center': jnp.array([1.0, 1.5, 2.0]), 'radius': 0.25},
    {'center': jnp.array([1.5, 2.0, 3.0]), 'radius': 0.3},
    {'center': jnp.array([0.5, 2.5, 1.5]), 'radius': 0.2},
    {'center': jnp.array([2.0, 3.0, 3.0]), 'radius': 0.3},
    {'center': jnp.array([0.0, 3.0, 3.0]), 'radius': 0.3},
]
quadrotor_sphere_radius = 0.45  # Adjust this to change the size

class DummyEqualityConstraint(AbstractConstraint):
    def constraint_fn(self, x, u, t):
        del x, u, t
        return jnp.empty(1)
    
class InequalityConstraintWithObstacles(AbstractConstraint):
    def constraint_fn(self, x, u, t):
        u_min = jnp.array([0., -10., -10., -10.])
        u_max = jnp.array([50., 10., 10., 10.])

        h_con = jnp.concatenate([u - u_max, u_min - u])
        h_con = jnp.where(t == T, jnp.zeros_like(h_con), h_con)

        pos = x[0:3]
        h_obs = []
        for obs in obstacles:
            obstacle_center = obs['center']
            obstacle_radius = obs['radius']
            dist_to_obstacle = jnp.dot(pos - obstacle_center, pos - obstacle_center)
            h = dist_to_obstacle - (obstacle_radius + quadrotor_sphere_radius)**2
            h_obs.append(h)

        h_con = jnp.concatenate([h_con, 
                                -jnp.array(h_obs)])

        return h_con

    def jacobian(self, x, u, t, *args):
        jac_x = jax.jacobian(self.constraint_fn)(x, u, t, *args)
        jac_x = jac_x[:,:-1] # hardcoded
        jac_u = jax.jacobian(self.constraint_fn, argnums=1)(x, u, t, *args)
        return jac_x, jac_u

In [4]:
from ilqrx.ocp import OptimalControlProblem
from ilqrx.solvers import CILQRSolver, CILQRSolverOptions

solver_options = CILQRSolverOptions(rho_init=10.0,
                                    opti_tol_init=1.0, rho_scaling=100.0,
                                    # constr_vio_tol_init=1.0,
                                    constr_vio_tol_init=0.01,
                                    max_iter=1000)

ocp = OptimalControlProblem(
    dynamic_model=quadrotor_model,
    cost_model=cost_model,
    equality_constraint_model=DummyEqualityConstraint(),
    inequality_constraint_model=InequalityConstraintWithObstacles(),
)

cilqr_solver = CILQRSolver(ocp, solver_options)

cilqr_solution = cilqr_solver.solve(
    X,
    U,
    verbose=True
)
X_opt = cilqr_solution.X


╔═══ Starting CILQR Optimization ═══╗
---
iter |      cost        |  lag_x_inf_norm  |  lag_u_inf_norm  |  defect_inf_norm  |      alpha
 1   |   1.026815e+04   |   1.571036e+02   |   2.691298e+00   |   5.118606e-02    |   5.000000e-01  
 2   |   1.216457e+04   |   8.733726e+01   |   6.862604e-01   |   7.571482e-03    |   1.000000e+00  
 3   |   1.210046e+04   |   1.661155e+01   |   8.444184e-02   |   2.125657e-04    |   1.000000e+00  
 4   |   1.209911e+04   |   8.553463e+00   |   3.943601e-02   |   6.174386e-05    |   1.000000e+00  
 5   |   1.209934e+04   |   4.219954e+00   |   1.960110e-02   |   1.278026e-05    |   1.000000e+00  
 6   |   1.209940e+04   |   2.899884e+00   |   1.573004e-02   |   5.127777e-06    |   1.000000e+00  
 7   |   1.209943e+04   |   1.988993e+00   |   1.187286e-02   |   2.049644e-06    |   1.000000e+00  
 8   |   1.209945e+04   |   1.459601e+00   |   9.434474e-03   |   1.066955e-06    |   1.000000e+00  
 9   |   1.209945e+04   |   1.067917e+00   |   7.15733

In [5]:
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import os
import numpy as np
import meshcat.geometry as geo
import meshcat.transformations as tf

# Get the path to the URDF file
# In notebooks, use os.getcwd() instead of __file__
current_dir = os.getcwd()
urdf_path = os.path.join(current_dir, "hector_description/robots/quadrotor_base.urdf")
mesh_dir = os.path.join(current_dir, "hector_description")

# Build Pinocchio models from the URDF with a floating base
# This adds a free-flyer joint (6 DOF: 3 for position, 3 for orientation)
model = pin.buildModelFromUrdf(urdf_path, pin.JointModelFreeFlyer())
collision_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.COLLISION, package_dirs=mesh_dir)
visual_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.VISUAL, package_dirs=mesh_dir)

# Create the visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)

# Start / connect to MeshCat (open=True launches a browser tab)
viz.initViewer(open=True)

# Load the robot geometry into the viewer
viz.loadViewerModel()

# Create a sphere geometry for the obstacle
for i, obs in enumerate(obstacles):
    obstacle_center = np.array(obs['center'])
    obstacle_radius = obs['radius']
    viz.viewer[f"obstacle_{i}"].set_object(
        geo.Sphere(obstacle_radius),
        geo.MeshLambertMaterial(color=0xff0000, opacity=0.5)  # Red semi-transparent
    )
    # Position the obstacle
    viz.viewer[f"obstacle_{i}"].set_transform(
        tf.translation_matrix(obstacle_center)
    )
    print(f"Added obstacle sphere at {obstacle_center} with radius {obstacle_radius}")

# Add a sphere around the quadrotor (safety/proximity zone)
viz.viewer["quadrotor_sphere"].set_object(
    geo.Sphere(quadrotor_sphere_radius),
    geo.MeshLambertMaterial(color=0x00ff00, opacity=0.2)  # Green semi-transparent wireframe
)
print(f"Added quadrotor safety sphere with radius {quadrotor_sphere_radius}")



# Setup MeshCat animation with playback controls
import meshcat.animation as anim

print("Setting up MeshCat animation with playback controls...")

# Create animation
animation = anim.Animation(default_framerate=int(1.0/dt))

# Prepare all configurations for the trajectory
configs = []
for state in X_opt:
    pos = np.array(state[:3])  # x, y, z
    quat = np.array(state[3:7])  # qw, qx, qy, qz (in JAX convention)
    
    # Convert to Pinocchio's quaternion convention: [qx, qy, qz, qw]
    q_pin = np.zeros(7)
    q_pin[:3] = pos  # position
    q_pin[3:7] = [quat[1], quat[2], quat[3], quat[0]]  # qx, qy, qz, qw
    configs.append(q_pin)

# Add frames to animation for each timestep
for frame_idx, q in enumerate(configs):
    # Update model data
    data = model.createData()
    visual_data = visual_model.createData()
    pin.forwardKinematics(model, data, q)
    pin.updateGeometryPlacements(model, data, visual_model, visual_data)
    
    # Set animation frame for each visual geometry
    with animation.at_frame(viz.viewer, frame_idx) as frame:
        for geom_idx, geom in enumerate(visual_model.geometryObjects):
            oMg = visual_data.oMg[geom_idx]
            
            # Get the visualizer path for this geometry
            geom_path = viz.getViewerNodeName(geom, pin.GeometryType.VISUAL)
            
            # Set transformation at this frame
            T = np.eye(4)
            T[:3, :3] = oMg.rotation
            T[:3, 3] = oMg.translation
            frame[geom_path].set_transform(T)
        
        pos = q[:3]
        frame["quadrotor_sphere"].set_transform(tf.translation_matrix(pos))

# Set the animation in the visualizer
viz.viewer.set_animation(animation, play=True, repetitions=1)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Added obstacle sphere at [1.  1.5 2. ] with radius 0.25
Added obstacle sphere at [1.5 2.  3. ] with radius 0.3
Added obstacle sphere at [0.5 2.5 1.5] with radius 0.2
Added obstacle sphere at [2. 3. 3.] with radius 0.3
Added obstacle sphere at [0. 3. 3.] with radius 0.3
Added quadrotor safety sphere with radius 0.45
Setting up MeshCat animation with playback controls...
Opening in existing browser session.
